In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import StandardScaler

from sqlalchemy import create_engine
import psycopg2

#### LOAD DATA TABLE PRODUCT

In [2]:
data_1 = pd.read_csv('products.csv')

#### LOAD DATA TABLE TRANSACTIONS

In [3]:
data_2 = pd.read_csv('transactions.csv')

#### CLEANING DATA

##### DATA 1

In [4]:
data_1 = data_1.dropna(subset=['product_id','product_name'])

In [5]:
data_1= data_1[(data_1['product_id'].str.contains('PROD'))].reset_index(drop=True)

In [6]:
data_1.loc[data_1['base_price'].str.contains(r'[a-zA-Z]', na=False),'base_price']=0

In [7]:
data_1['base_price']=data_1['base_price'].astype(float)

In [8]:
str_columns = ['product_name', 'category', 'manufacturer']

In [9]:
for i in str_columns :
    data_1[i] = data_1[i].fillna('-')
    data_1[i] = data_1[i].str.lower()
    data_1[i] = data_1[i].astype(str)

In [10]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    188 non-null    object 
 1   product_name  188 non-null    object 
 2   category      188 non-null    object 
 3   manufacturer  188 non-null    object 
 4   base_price    187 non-null    float64
dtypes: float64(1), object(4)
memory usage: 7.5+ KB


In [11]:
data_1

,product_id,product_name,category,manufacturer,base_price
0,PROD-a7e8c7dd,books item 72,books,ecotech,475.85
1,PROD-a9a7608e,sports & outdoors item 467,sports & outdoors,globalbrands,0.00
2,PROD-38e2b395,home & kitchen item 100,home & kitchen,-683,235.03
3,PROD-048762fb,home & kitchen item 344,home & kitchen,globalbrands_noise_onto,20.09
4,PROD-1f031ffb,-605,books,fashionhouse,99.87
...,...,...,...,...,...
183,PROD-9caba23d,electronics item 404,electronics,ecotech,181.11
184,PROD-c23e242f_project,clothing item 298,clot,sportspro,463.10
185,PROD-48bda8f6,286,books,techgiant,199.19
186,PROD-c2fc02b2,-969,home & kitchen,globalbrands,487.10


##### DATA 2

In [12]:
data_2 = data_2.dropna(subset=['transaction_id','customer_id','product_id'])

In [13]:
data_2.loc[~(data_2['product_id'].str.contains('PROD')),'product_id']='other'

In [14]:
data_2.loc[~(data_2['transaction_id'].str.contains('TRX')),'transaction_id']='TRX'+'_'+data_2['transaction_id']

In [15]:
columns = ['quantity', 'total_price']
for i in columns :
    data_2[i]=data_2[i].astype('str')
    data_2[i]=data_2[i].fillna('0')
    data_2[i]=data_2[i].str.strip(' ')
    data_2[i]=data_2[i].str.split('_').str[0]
    data_2[i]=data_2[i].str.replace('nan','0')
    data_2[i]=data_2[i].str.replace('MISSING','0')
    data_2.loc[data_2[i]=='n',i]='0'

In [16]:
data_2['sales_channel']=data_2['sales_channel'].str.strip(' ')
data_2['sales_channel']=data_2['sales_channel'].str.lower()
data_2['sales_channel']=data_2['sales_channel'].fillna('other')
data_2['sales_channel']=data_2['sales_channel'].str.split('_').str[0]
data_2['sales_channel']=data_2['sales_channel'].str.replace('nan','other')
data_2['sales_channel']=data_2['sales_channel'].str.replace('missing','other')
data_2.loc[data_2['sales_channel']=='n','sales_channel']='other'

In [17]:
data_2.loc[~(data_2['sales_channel'].str.contains('mobile|in-store|online')),'sales_channel']='other'

In [20]:
data_2['quantity']=data_2['quantity'].astype(float)
data_2['total_price']=data_2['total_price'].astype(float)

In [21]:
data_2['sales_channel']=data_2['sales_channel'].fillna('-')
data_2['sales_channel']=data_2['sales_channel'].str.lower()

In [22]:
data_2.loc[data_2['sale_date'].str.contains(r'[a-zA-Z]', na=False),'sale_date']='1997-01-01'

In [23]:
data_2['sale_date']=pd.to_datetime(data_2['sale_date'], errors='coerce')

In [24]:
data_2['sale_date']=data_2['sale_date'].fillna('1997-01-01')

In [25]:
data_2['sale_date']=pd.to_datetime(data_2['sale_date'])

In [26]:
# Remove transactions where quantity or total_price is less than 0
data_2 = data_2[(data_2['quantity'] >= 0) & (data_2['total_price'] >= 0)]

In [44]:
data_2.loc[~data_2['customer_id'].str.contains('CUST'),'customer_id']='other'

### Total revenue per product

In [27]:
#GROUP BY PRODUCT
data_grouped = data_2.groupby('product_id').agg({'total_price': 'sum', # Sum sales for each product
                                                                'quantity': 'sum'     # Sum qty sold or each product
                                                                }).reset_index()

In [28]:
data_grouped

,product_id,total_price,quantity
0,PROD-0,3478.14,30.0
1,PROD-004aea84,25464.62,176.0
2,PROD-01aaa947,48788.88,113.0
3,PROD-02362c25,18087.08,99.0
4,PROD-048762fb,1988.91,99.0
...,...,...,...
212,PROD-fc92cad8,27263.74,165.0
213,PROD-fda1424b,27391.60,93.0
214,PROD-fe031249,14760.79,123.0
215,PROD-fe40d8d3,23175.21,131.0


In [29]:
# Merge with product data to get additional information like product_name and base_price
data_grouped = data_grouped.merge(data_1, on='product_id',how='left')

In [30]:
for i in str_columns:
    data_grouped[i] = data_grouped[i].fillna('other')

In [31]:
data_grouped['base_price'] = data_grouped['base_price'].fillna(0)

In [32]:
data_grouped = data_grouped.sort_values(by=['total_price', 'quantity'], ascending=False).reset_index(drop=True)

In [33]:
data_grouped

,product_id,total_price,quantity,product_name,category,manufacturer,base_price
0,other,433405.46,1855.0,other,other,other,0.00
1,PROD-206a28be,85260.90,175.0,clothing item 127,-,luxurygoods,481.70
2,PROD-ef9b7568,79237.60,161.0,electronics item 777,electronics,-763,483.16
3,PROD-91e8e92e,74931.68,198.0,electronics item 973,electronics,189,374.66
4,PROD-26037271,74483.64,154.0,beauty & personal care item 700,beauty & personal care,ecotech,0.00
...,...,...,...,...,...,...,...
227,PROD-e1ae7b0a,2037.40,122.0,electronics item 518,electronics,luxurygoods,16.70
228,PROD-4b745662,2012.43,111.0,other,other,other,0.00
229,PROD-048762fb,1988.91,99.0,home & kitchen item 344,home & kitchen,globalbrands_noise_onto,20.09
230,PROD-e,1205.84,25.0,other,other,other,0.00


#### Sales performance by channel

In [37]:
#GROUP BY PRODUCT
data_grouped_2 = data_2.groupby('sales_channel').agg({'total_price': 'sum', # Sum sales for each product
                                                 'quantity': 'sum'     # Sum qty sold or each product
                                                    }).reset_index()

In [38]:
data_grouped_2

,sales_channel,total_price,quantity
0,in-store,2150127.92,8358.0
1,mobile app,2080648.33,8572.0
2,online,2144082.16,8572.0
3,other,582017.42,2480.0


#### Customer purchase frequency

In [45]:
#GROUP BY PRODUCT
data_grouped_3 = data_2.groupby('customer_id').agg({'transaction_id': 'nunique', # Sum sales for each product
                                                    }).reset_index()

In [48]:
data_grouped_3 = data_grouped_3.sort_values(by=['transaction_id'], ascending=False).reset_index(drop=True)

In [51]:
data_grouped_3

,customer_id,transaction_id
0,other,721
1,CUST-4,20
2,CUST-c,16
3,CUST-d,15
4,CUST-5,14
...,...,...
8371,CUST-54844586,1
8372,CUST-5481a7c3,1
8373,CUST-547fc471,1
8374,CUST-5473f973_laugh,1
